In [1]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Step 1: Load PDF
loader = PyMuPDFLoader("data/Huy_Bui_Resume.pdf")  # Replace with your PDF path
documents = loader.load()
print("Number of words:", len(documents[0].page_content.split(" ")))

# Step 2: Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

# Step 3: Use HuggingFace Embeddings (open source)
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False} #False Euclidean, True cosine similarity
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Step 4: Create FAISS vector index
vector_store = FAISS.from_documents(chunks, hf)

# Step 5: Save the index locally
vector_store.save_local("faiss_index_open")
print("Indexing complete and saved to 'faiss_index_open'")


Number of words: 360
Indexing complete and saved to 'faiss_index_open'


In [2]:
# Extract raw texts
texts = [chunk.page_content for chunk in chunks[:3]]  # First 3 chunks

# Get embeddings
vectors = hf.embed_documents(texts)

for i, (text, vec) in enumerate(zip(texts, vectors)):
    print(f"\n🔢 Embedding {i+1}:")
    print(f"Text: {text[:100]}...")
    print(f"Vector (first 5 dims): {vec[:5]}")
    print(f"Vector length: {len(vec)}")



🔢 Embedding 1:
Text: Huy Bui
williamhuybui@gmail.com | linkedin.com/in/huy-bui-ds
Experience
Publicis Groupe
Remote
Senio...
Vector (first 5 dims): [-0.009209658950567245, 0.09017603099346161, -0.02930491603910923, -0.06725069135427475, 0.003979860804975033]
Vector length: 768

🔢 Embedding 2:
Text: – Built an automated data pipeline leveraging Python, SQL, BigQuery, AWS Lambda, S3, and EventBridge...
Vector (first 5 dims): [0.03445270285010338, 0.089835524559021, -0.04970095679163933, -0.06849135458469391, 0.005093281622976065]
Vector length: 768

🔢 Embedding 3:
Text: Research Data Scientist
April 2020 – June 2022
– Lead author of the research Machine Learning Applic...
Vector (first 5 dims): [-0.06988285481929779, 0.020990032702684402, -0.006211338099092245, -0.03602779284119606, -0.020267002284526825]
Vector length: 768


In [3]:
results = vector_store.similarity_search_with_score(
    "Mentor ?", k=4
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.372315] Mentor | Techsphere
August 2022 - August 2024
– Helped Vietnamese professionals with mock interviews in data science and software engineering.
1st Place Team at Glasstire DataHack | Rice University
December 2019
3rd Place Team at TAMU Hack | Texas A&M University
January 2020 [{'producer': 'pdfTeX-1.40.24', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-05-19T01:02:47+00:00', 'source': 'data/Huy_Bui_Resume.pdf', 'file_path': 'data/Huy_Bui_Resume.pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-05-19T01:02:47+00:00', 'trapped': '', 'modDate': 'D:20250519010247Z', 'creationDate': 'D:20250519010247Z', 'page': 0}]
* [SIM=1.491172] clients and reducing manual processing time by 30%.
Texas A&M University
College Station, Texas
Teaching Assistant
August 2017 – May 2019
– Taught Python, Matlab, Calculus I, II, Differential Equations, and Discrete Mathematics to engineering students,
assisting with l